In [ ]:
import time
import copy
import pygame
import sys

mutat = False

class Joc:
    """
    Clasa care defineste jocul. Se va schimba de la un joc la altul.
    """
    JMIN = None
    JMAX = None
    GOL = "#"
    NR_LINII = None
    NR_COLOANE = None
    scor_maxim = 0


    @classmethod
    def initializeaza(cls, display, NR_LINII=8, NR_COLOANE=8, dim_celula=50):
        
        cls.NR_LINII = NR_LINII
        cls.NR_COLOANE = NR_COLOANE
        
        cls.a_img = pygame.image.load("piesa_alb.png")
        cls.a_img = pygame.transform.scale(cls.a_img, (dim_celula, dim_celula))
        cls.n_img = pygame.image.load("piesa_negru.png")
        cls.n_img = pygame.transform.scale(cls.n_img, (dim_celula, dim_celula))
        cls.ra_img = pygame.image.load("rege_alb.png")
        cls.ra_img = pygame.transform.scale(cls.ra_img,(dim_celula,dim_celula))
        cls.rn_img = pygame.image.load("rege_negru.png")
        cls.rn_img = pygame.transform.scale(cls.rn_img,(dim_celula,dim_celula))
        
        cls.display = display
        cls.dim_celula = dim_celula

        # calculare scor maxim
        cls.scor_maxim = 2*4*3 # numarul maxim de piese regi pe care le poate avea un jucator. consideram ca un rege valoreaza de doua ori mai mult decat o piesa normala

        cls.celuleGrid = []  # este lista cu patratelele din grid
        
        for linie in range(NR_LINII):
            for coloana in range(NR_COLOANE):
                patr = pygame.Rect(coloana * (dim_celula + 1), linie * (dim_celula + 1), dim_celula, dim_celula)
                cls.celuleGrid.append(patr)
    
    def __init__(self, matr=None):
        # creez proprietatea ultima_mutare # (l,c)
        self.ultima_mutare = None

        if matr:
            # e data tabla, deci suntem in timpul jocului
            self.matr = matr
        else:
            # nu e data tabla deci suntem la initializare 
            self.matr = [
                [self.__class__.GOL,"a",self.__class__.GOL,"a",self.__class__.GOL,"a",self.__class__.GOL,"a"],
                ["a",self.__class__.GOL,"a",self.__class__.GOL,"a",self.__class__.GOL,"a",self.__class__.GOL],
                [self.__class__.GOL,"a",self.__class__.GOL,"a",self.__class__.GOL,"a",self.__class__.GOL,"a"],
                [self.__class__.GOL,self.__class__.GOL,self.__class__.GOL,self.__class__.GOL,self.__class__.GOL,self.__class__.GOL,self.__class__.GOL,self.__class__.GOL],
                [self.__class__.GOL,self.__class__.GOL,self.__class__.GOL,self.__class__.GOL,self.__class__.GOL,self.__class__.GOL,self.__class__.GOL,self.__class__.GOL],
                ["n",self.__class__.GOL,"n",self.__class__.GOL,"n",self.__class__.GOL,"n",self.__class__.GOL],
                [self.__class__.GOL,"n",self.__class__.GOL,"n",self.__class__.GOL,"n",self.__class__.GOL,"n"],
                ["n",self.__class__.GOL,"n",self.__class__.GOL,"n",self.__class__.GOL,"n",self.__class__.GOL]
            ]

    def deseneaza_grid(self, marcaj=None):

        for linie in range(self.__class__.NR_LINII):
            for coloana in range(self.__class__.NR_COLOANE):
                # desenarea tabla 
                if (coloana+linie) % 2 == 1: 
                    culoare = (102, 70, 46) #coloram cu maro patratele 
                else:
                    culoare = (227, 214, 191)#coloram cu bej
                    
                # desenare patratica marcata
                if(marcaj):
                    if(marcaj[0] == linie and marcaj[1] == coloana):
                        culoare = (153, 142, 142) #daca selectam piesa patratul se coloreaza cu gri
                pygame.draw.rect(self.__class__.display, culoare, self.__class__.celuleGrid[linie * self.__class__.NR_COLOANE + coloana])  # alb = (255,255,255)
                
                if self.matr[linie][coloana] == "a":
                    self.__class__.display.blit(self.__class__.a_img,
                        (coloana * (self.__class__.dim_celula + 1),linie * (self.__class__.dim_celula + 1)))
                
                elif self.matr[linie][coloana] == "n":
                    self.__class__.display.blit(self.__class__.n_img,
                        (coloana * (self.__class__.dim_celula + 1), linie * (self.__class__.dim_celula + 1)))
                    
                elif self.matr[linie][coloana] == "A":
                    self.__class__.display.blit(self.__class__.ra_img, 
                        (coloana *(self.__class__.dim_celula+1),linie * (self.__class__.dim_celula +1)))
                    
                elif self.matr[linie][coloana] == "N":
                    self.__class__.display.blit(self.__class__.rn_img, 
                        (coloana *(self.__class__.dim_celula+1), linie * (self.__class__.dim_celula +1)))
        pygame.display.update()

    @classmethod
    def jucator_opus(cls, jucator):
        return cls.JMAX if jucator == cls.JMIN else cls.JMIN


    def parcurgerea(self,culoare):
        if culoare == 'n' or culoare == 'N':
            culoareopus = 'a'
        else:
            culoareopus = 'n'
        if(self.check_possibilities(culoareopus)== 0):
            return True
        else:
            return False 
        # ei bine se pare ca check posi e buna si pentru cazul in care nu mai are piese. pt ca atunci cand o culoare nu mai are piese implicit nu are mutari deci va ramane zero
    
    def check_possibilities(self,culoare):
        """Functie care verifica pentru un jucator daca mai are mutari posibile
        """
        #folosim variabilele culoareR si culoareP in care pastram culoarea piesei (culoareR pt rege si culoareP pentru piesa normala)
        nr_mutari = 0
        if (culoare == 'A' or culoare =='a' ): #  Daca piesa este alba (rege sau piesa normala) 
            culoareR = 'A'
            culoareP = 'a'
        else:
            culoareR = 'N'
            culoareP = 'n'
            
        for i in range(self.__class__.NR_LINII):
            for j in range(self.__class__.NR_COLOANE):
                
                if self.matr[i][j] == culoareP: # daca este piesa simpla
                    
                    if (culoareP == 'a'):
                        # doar in jos
                        directii = [(1,1), (1,-1)]
                    else:
                        # doar in sus
                        directii = [(-1, -1), (-1, 1)]
                        
                    # Daca piesa este simpla si neagra, aceasta se poate muta doar pe diagonalele de sus, 
                    # iar daca piesa este simpla si alba aceasta se poate muta doar in diagonalele de jos. 
                    # Regii pot fi mutati pe orice diagonala. Verificam daca avem putem face mutare, adica 
                    # daca avem patratelul de pe diagonala liber, sau daca pe diagonala este culoarea opusa 
                    # urmata de spatiu gol.

                    
                    for directie in directii:
                        
                        mutare = (i+directie[0],j+directie[1])
                        
                        if (mutare[0] < 0 or mutare[0] >= self.__class__.NR_LINII or mutare[1] < 0 or mutare[1] >= self.__class__.NR_COLOANE):
                            continue
                            
                        if(self.matr[mutare[0]][mutare[1]] != self.__class__.GOL and self.matr[mutare[0]][mutare[1]] != culoareP and self.matr[mutare[0]][mutare[1]] != culoareR):
                            #adica daca in diagonala e piesa opusa
                            
                            mutare1 = (mutare[0]+directie[0],mutare[1]+directie[1])
                            if (mutare1[0] < 0 or mutare1[0] >= self.__class__.NR_LINII or mutare1[1] < 0 or mutare1[1] >= self.__class__.NR_COLOANE):
                                continue
                            if(self.matr[mutare1[0]][mutare1[1]] == self.__class__.GOL):
                                #inseamna ca se poate face salt
                                nr_mutari +=1
                                
                        if (self.matr[mutare[0]][mutare[1]] == self.__class__.GOL):
                            # inseamna ca se poate face mutarea aici
                            nr_mutari +=1

                elif self.matr[i][j] == culoareR: # Daca este rege
                    directii = [ (1, 1), (-1, -1), (1, -1), (-1, 1)]
                    for directie in directii:
                        mutare = (i+directie[0],j+directie[1])
                        if (mutare[0] < 0 or mutare[0] >= self.__class__.NR_LINII or mutare[1] < 0 or mutare[1] >= self.__class__.NR_COLOANE):
                            continue
                        if(self.matr[mutare[0]][mutare[1]] != self.__class__.GOL and self.matr[mutare[0]][mutare[1]] != culoareP and self.matr[mutare[0]][mutare[1]] != culoareR):
                            #daca in diagonala e piesa opusa

                            mutare1 = (mutare[0]+directie[0],mutare[1]+directie[1])
                            if (mutare1[0] < 0 or mutare1[0] >= self.__class__.NR_LINII or mutare1[1] < 0 or mutare1[1] >= self.__class__.NR_COLOANE):
                                continue
                            if(self.matr[mutare1[0]][mutare1[1]] == self.__class__.GOL):
                                #se poate face mutare
                                nr_mutari +=1

                        if (self.matr[mutare[0]][mutare[1]] == self.__class__.GOL):
                            #se poate face mutarea aici
                            nr_mutari +=1
        return nr_mutari

    def final(self):
        """Returneaza castigatorul sau "remiza" daca jocul s-a incheiat,
        altfel returneaza False.
        """
        
        if not self.ultima_mutare:
            return False
        um = self.ultima_mutare
        culoare = self.matr[um[0]][um[1]]
        if (self.parcurgerea(culoare)):
            #inseamna ca adversarul nu mai are mutari dar trb verificat si daca eu am mutari
            if culoare == 'n' or culoare == 'N':
                culoareopus = 'a'
            else:
                culoareopus = 'n'
            if(self.parcurgerea(culoareopus)):
                return "remiza"
            else:
                return culoare
        else:
            return False


        # Daca ultima mutare face ca oponentul sa nu mai aiba piese pe tabla atunci castiga, 
        # daca ultima mutare face ca oponentul sa nu mai aiba mutari valide atunci castiga, 
        # daca niciun jucator nu mai are mutari valide atunci avem remiza
    
    def mutari(self, culoare):
        """Functia este pentru calculator pentru a genera toate mutarile posibile 
        returneaza o lista de jocuri care se pot obtine din o mutare.
        Mai intai trebuie sa cautam daca putem efectua sarituri. daca se pot efectua sarituri in l_mutari punem doar sarituri. daca nu se pot efectua parcurgem din nou si punem toate posibile
        """
        if (culoare == 'A' or culoare =='a' ): #  Daca piesa este alba (rege sau piesa normala) 
            culoareR = 'A'
            culoareP = 'a'
        else:
            culoareR = 'N'
            culoareP = 'n'
            
        l_mutari = []
        
        global mutat
        mutat = False
        
        for i in range(self.__class__.NR_LINII):
            for j in range(self.__class__.NR_COLOANE):
                #mai intai trebuie sa gasim piesa
                if self.matr[i][j] == culoareP :
                    if (culoareP == "n"):
                        #doar in sus
                        directii = [(-1, -1),(-1, 1)]
                        linie_rege = 0
                        
                    else:
                        #doar in jos
                        directii = [(1,1),(1,-1)]
                        linie_rege = 7
                        
                    for directie in directii:
                        mutare = (i+directie[0],j+directie[1])
                        if (mutare[0] < 0 or mutare[0] >= self.__class__.NR_LINII or mutare[1] < 0 or mutare[1] >= self.__class__.NR_COLOANE):
                            continue
                        if(self.matr[mutare[0]][mutare[1]] != self.__class__.GOL and self.matr[mutare[0]][mutare[1]] != culoareP and self.matr[mutare[0]][mutare[1]] != culoareR): 
                            #adica daca in diagonala e piesa opusa
                           
                            mutare1 = (mutare[0]+directie[0],mutare[1]+directie[1])
                            if (mutare1[0] < 0 or mutare1[0] >= self.__class__.NR_LINII or mutare1[1] < 0 or mutare1[1] >= self.__class__.NR_COLOANE):
                                continue
                            if(self.matr[mutare1[0]][mutare1[1]] == self.__class__.GOL):
                                #inseamna ca se poate face salt
                                mutat = True
                                matr_tabla_noua = copy.deepcopy(self.matr)
                                matr_tabla_noua[mutare[0]][mutare[1]] = self.__class__.GOL
                                if(linie_rege == mutare1[0]):
                                    matr_tabla_noua[mutare1[0]][mutare1[1]] = culoareR
                                else:
                                    matr_tabla_noua[mutare1[0]][mutare1[1]] = matr_tabla_noua[i][j] 
                                matr_tabla_noua[i][j] = self.__class__.GOL
                                joc_nou = Joc(matr_tabla_noua)
                                joc_nou.ultima_mutare = (mutare1[0],mutare1[1])
                                l_mutari.append(joc_nou)
                                
                elif self.matr[i][j] == culoareR :
                    directii = [(1, 1), (-1, -1), (1, -1), (-1, 1)]
                    for directie in directii:
                        mutare = (i+directie[0],j+directie[1])
                        if (mutare[0] < 0 or mutare[0] >= self.__class__.NR_LINII or mutare[1] < 0 or mutare[1] >= self.__class__.NR_COLOANE):
                            continue
                        if(self.matr[mutare[0]][mutare[1]] != self.__class__.GOL and self.matr[mutare[0]][mutare[1]] != culoareP and self.matr[mutare[0]][mutare[1]] != culoareR):
                            #adica daca in diagonala e piesa opusa
                            mutare1 = (mutare[0]+directie[0],mutare[1]+directie[1])
                            if (mutare1[0] < 0 or mutare1[0] >= self.__class__.NR_LINII or mutare1[1] < 0 or mutare1[1] >= self.__class__.NR_COLOANE):
                                continue
                            if(self.matr[mutare1[0]][mutare1[1]] == self.__class__.GOL):
                                #inseamna ca se poate face salt
                                mutat = True
                                matr_tabla_noua = copy.deepcopy(self.matr)
                                matr_tabla_noua[i][j] = self.__class__.GOL
                                matr_tabla_noua[mutare[0]][mutare[1]] = self.__class__.GOL
                                matr_tabla_noua[mutare1[0]][mutare1[1]] = culoareR
                                joc_nou = Joc(matr_tabla_noua)
                                joc_nou.ultima_mutare = (mutare1[0],mutare1[1])
                                l_mutari.append(joc_nou)
        if (not mutat):
            for i in range(self.__class__.NR_LINII):
                for j in range(self.__class__.NR_COLOANE):
                    #mai intai trebuie sa gasim piesa
                    if self.matr[i][j] == culoareP :
                        #mai intai trebuie sa mi gasesc piesa
                        if (culoareP == "n"):
                            #adica pot sa ma duc doar in sus
                            directii = [(-1, -1), (-1, 1)]
                            linie_rege = 0
                        else:
                            #adica pot sa ma duc doar in jos
                            directii = [(1,1), (1,-1)]
                            linie_rege = 7
                            
                        for directie in directii:
                            mutare = (i+directie[0],j+directie[1])
                            if (mutare[0] < 0 or mutare[0] >= self.__class__.NR_LINII or mutare[1] < 0 or mutare[1] >= self.__class__.NR_COLOANE):
                                continue
                            if (self.matr[mutare[0]][mutare[1]] == self.__class__.GOL):
                                matr_tabla_noua = copy.deepcopy(self.matr)
                                
                                if(linie_rege == mutare[0]):
                                    matr_tabla_noua[mutare[0]][mutare[1]] = culoareR
                                else:
                                    matr_tabla_noua[mutare[0]][mutare[1]] = matr_tabla_noua[i][j]
                                matr_tabla_noua[i][j] = self.__class__.GOL
                                joc_nou = Joc(matr_tabla_noua)
                                joc_nou.ultima_mutare = (mutare[0],mutare[1])
                                l_mutari.append(joc_nou)

                    elif self.matr[i][j] == culoareR :
                        directii = [(1, 1), (-1, -1), (1, -1), (-1, 1)]
                        for directie in directii:
                            mutare = (i+directie[0],j+directie[1])
                            if (mutare[0] < 0 or mutare[0] >= self.__class__.NR_LINII or mutare[1] < 0 or mutare[1] >= self.__class__.NR_COLOANE):
                                continue
                            if (self.matr[mutare[0]][mutare[1]] == self.__class__.GOL):
                                matr_tabla_noua = copy.deepcopy(self.matr)
                                matr_tabla_noua[i][j] = self.__class__.GOL
                                matr_tabla_noua[mutare[0]][mutare[1]] = culoareR
                                joc_nou = Joc(matr_tabla_noua)
                                joc_nou.ultima_mutare = (mutare[0],mutare[1])
                                l_mutari.append(joc_nou)

        return l_mutari

    def val_piese(self, culoare):
        """Functie folosita pentru estimarea scorului unei stari.
        """
        if (culoare == 'A' or culoare =='a' ): #  Daca piesa este alba (rege sau piesa normala) 
            culoareR = 'A'
            culoareP = 'a'
        else:
            culoareR = 'N'
            culoareP = 'n'
            
        # primul mod de estimare scor
        val = 0
        for i in range(self.__class__.NR_LINII):
            for j in range(self.__class__.NR_COLOANE):
                # daca este rege are atunci are valoarea 2, daca este piesa normala are valoarea 1
                if(self.matr[i][j] == culoareP):
                    val+=1
                elif(self.matr[i][j] == culoareR):
                    val+=2
                    
        # al doilea mod de estimare scor:
        # consideram ca daca piesa se afla in jumatatea oponentului are valoare mai mare
        # astfel, piesele aflate in jumatatea oponentului vor avea 2 puncte, 
        # piesele simple in jumatatea proprie vor avea 1 punct, iar regii vor avea 3 puncte

        # if (culoareP == 'n'):
        #     val = 0
        #     for i in range(self.__class__.NR_LINII):
        #         for j in range(self.__class__.NR_COLOANE):
        #             if(self.matr[i][j]==culoareP):
        #                 if(i<4):
        #                     val+=2
        #                 else:
        #                     val+=1
        #             elif(self.matr[i][j]==culoareR):
        #                 val+=3
        # elif(culoareP =='a'):
        #     val = 0
        #     for i in range(self.__class__.NR_LINII):
        #         for j in range(self.__class__.NR_COLOANE):
        #             if(self.matr[i][j]==culoareP):
        #                 if(i>=4):
        #                     val+=2
        #                 else:
        #                     val+=1
        #             elif(self.matr[i][j]==culoareR):
        #                 val+=3

            
        return val

    def estimeaza_scor(self, adancime):
        t_final = self.final()
        # if (adancime==0):
        if t_final == self.__class__.JMAX:
            return self.__class__.scor_maxim + adancime
        elif t_final == self.__class__.JMIN:
            return -self.__class__.scor_maxim - adancime
        elif t_final == "remiza":
            return 0
        else:
            # returnam diferenta de valori de piese pentru jucatori
            return self.val_piese(self.__class__.JMAX) - self.val_piese(self.__class__.JMIN)

    def sirAfisare(self):
        sir = "  |"
        sir += " ".join([chr(i) for i in range(97,105)]) + "\n"
        sir += "-" * (self.NR_COLOANE + 1) * 2 + "\n"
        sir += "\n".join([str(i) + " |" + " ".join([str(x) for x in self.matr[i]]) for i in range(len(self.matr))])
        return sir

    def __str__(self):
        return self.sirAfisare()

    def __repr__(self):
        return self.sirAfisare()

class Stare:
    """
    Clasa folosita de algoritmii minimax si alpha-beta
    Are ca proprietate tabla de joc
    Functioneaza cu conditia ca in cadrul clasei Joc sa fie definiti JMIN si JMAX (cei doi jucatori posibili)
    De asemenea cere ca in clasa Joc sa fie definita si o metoda numita mutari() care ofera lista cu configuratiile posibile in urma mutarii unui jucator
    """

    def __init__(self, tabla_joc, j_curent, adancime, parinte=None, scor=None):
        self.tabla_joc = tabla_joc
        self.j_curent = j_curent

        # adancimea in arborele de stari
        self.adancime = adancime

        # scorul starii (daca e finala) sau al celei mai bune stari-fiice (pentru jucatorul curent)
        self.scor = scor

        # lista de mutari posibile din starea curenta
        self.mutari_posibile = []

        # cea mai buna mutare din lista de mutari posibile pentru jucatorul curent
        self.stare_aleasa = None

    def mutari(self):
        l_mutari = self.tabla_joc.mutari(self.j_curent)
        juc_opus = Joc.jucator_opus(self.j_curent)
        l_stari_mutari = [
            Stare(mutare, juc_opus, self.adancime - 1, parinte=self)
            for mutare in l_mutari
        ]

        return l_stari_mutari

    def options(self,poz):
        lista =[]
        
        # mai intai trebuie sa identificam daca suntem alb sau negru
        # apoi trebuie verificat daca conform chestiilor unde poate merge
        culoareP = self.j_curent
        culoareR = chr(ord(culoareP)-32)
        mutat = False
        
        if(culoareP == "n"):
            if(self.tabla_joc.matr[poz[0]][poz[1]] == culoareP ):
                directii = [(-1, -1),(-1, 1)]
                for directie in directii:
                        mutare = (poz[0]+directie[0],poz[1]+directie[1])
                        if (mutare[0] < 0 or mutare[0] >= Joc.NR_LINII or mutare[1] < 0 or mutare[1] >= Joc.NR_COLOANE):
                            continue
                        if(self.tabla_joc.matr[mutare[0]][mutare[1]] != Joc.GOL and self.tabla_joc.matr[mutare[0]][mutare[1]] != culoareP and self.tabla_joc.matr[mutare[0]][mutare[1]] != culoareR):
                            #adica daca in diagonala e piesa opusa
                            mutare1 = (mutare[0]+directie[0],mutare[1]+directie[1])
                            if (mutare1[0] < 0 or mutare1[0] >= Joc.NR_LINII or mutare1[1] < 0 or mutare1[1] >= Joc.NR_COLOANE):
                                continue
                            if(self.tabla_joc.matr[mutare1[0]][mutare1[1]] == Joc.GOL):
                                mutat = True
                                lista.append(mutare1)
                                
            elif (self.tabla_joc.matr[poz[0]][poz[1]] == culoareR):
                directii = [ (1, 1), (-1, -1), (1, -1), (-1, 1)]
                for directie in directii:
                    mutare = (poz[0]+directie[0],poz[1]+directie[1])
                    if (mutare[0] < 0 or mutare[0] >= Joc.NR_LINII or mutare[1] < 0 or mutare[1] >= Joc.NR_COLOANE):
                        continue
                    if(self.tabla_joc.matr[mutare[0]][mutare[1]] != Joc.GOL and self.tabla_joc.matr[mutare[0]][mutare[1]] != culoareP and self.tabla_joc.matr[mutare[0]][mutare[1]] != culoareR):
                        #adica daca in diagonala e piesa opusa
                        mutare1 = (mutare[0]+directie[0],mutare[1]+directie[1])
                        if (mutare1[0] < 0 or mutare1[0] >= Joc.NR_LINII or mutare1[1] < 0 or mutare1[1] >= Joc.NR_COLOANE):
                            continue
                        if(self.tabla_joc.matr[mutare1[0]][mutare1[1]] == Joc.GOL):
                            mutat = True
                            lista.append(mutare1)         
            if(not mutat):
                if(self.tabla_joc.matr[poz[0]][poz[1]] == culoareP ):
                    directii = [(-1, -1),(-1, 1)]
                    for directie in directii:
                            mutare = (poz[0]+directie[0],poz[1]+directie[1])
                            if (mutare[0] < 0 or mutare[0] >= Joc.NR_LINII or mutare[1] < 0 or mutare[1] >= Joc.NR_COLOANE):
                                continue
                            if(self.tabla_joc.matr[mutare[0]][mutare[1]] == Joc.GOL):
                                lista.append(mutare)
                elif(self.tabla_joc.matr[poz[0]][poz[1]] == culoareR):
                    directii = [(1, 1), (-1, -1), (1, -1), (-1, 1)]
                    for directie in directii:
                            mutare = (poz[0]+directie[0],poz[1]+directie[1])
                            if (mutare[0] < 0 or mutare[0] >= Joc.NR_LINII or mutare[1] < 0 or mutare[1] >= Joc.NR_COLOANE):
                                continue
                            if(self.tabla_joc.matr[mutare[0]][mutare[1]] == Joc.GOL):
                                lista.append(mutare)
                        
        elif(culoareP == "a"):
            if(self.tabla_joc.matr[poz[0]][poz[1]] == culoareP):
                perechi_directii = [(1,1),(1,-1)]
                for directie in perechi_directii:
                        mutare = (poz[0]+directie[0],poz[1]+directie[1])
                        if (mutare[0] < 0 or mutare[0] >= Joc.NR_LINII or mutare[1] < 0 or mutare[1] >= Joc.NR_COLOANE):
                            continue
                        if(self.tabla_joc.matr[mutare[0]][mutare[1]] != Joc.GOL and self.tabla_joc.matr[mutare[0]][mutare[1]] != culoareP and self.tabla_joc.matr[mutare[0]][mutare[1]] != culoareR):
                            #adica daca in diagonala e piesa opusa
                            mutare1 = (mutare[0]+directie[0],mutare[1]+directie[1])
                            if (mutare1[0] < 0 or mutare[0] >= Joc.NR_LINII or mutare1[1] < 0 or mutare1[1] >= Joc.NR_COLOANE):
                                continue
                            if(self.tabla_joc.matr[mutare1[0]][mutare1[1]] == Joc.GOL):
                                mutat = True
                                lista.append(mutare1)

            elif (self.tabla_joc.matr[poz[0]][poz[1]] == culoareR):
                directii = [(1, 1), (-1, -1), (1, -1), (-1, 1)]
                for directie in directii:
                    mutare = (poz[0]+directie[0],poz[1]+directie[1])
                    if (mutare[0] < 0 or mutare[0] >= Joc.NR_LINII or mutare[1] < 0 or mutare[1] >= Joc.NR_COLOANE):
                        continue
                    if(self.tabla_joc.matr[mutare[0]][mutare[1]] != Joc.GOL and self.tabla_joc.matr[mutare[0]][mutare[1]] != culoareP and self.tabla_joc.matr[mutare[0]][mutare[1]] != culoareR):
                        #adica daca in diagonala e piesa opusa
                        mutare1 = (mutare[0]+directie[0],mutare[1]+directie[1])
                        if (mutare1[0] < 0 or mutare1[0] >= Joc.NR_LINII or mutare1[1] < 0 or mutare1[1] >= Joc.NR_COLOANE):
                            continue
                        if(self.tabla_joc.matr[mutare1[0]][mutare1[1]] == Joc.GOL):
                            mutat = True
                            lista.append(mutare1)
            if(not mutat):
                if(self.tabla_joc.matr[poz[0]][poz[1]] == culoareP ):
                    directii = [(1,1),(1,-1)]
                    for directie in directii:
                            mutare = (poz[0]+directie[0],poz[1]+directie[1])
                            if (mutare[0] < 0 or mutare[0] >= Joc.NR_LINII or mutare[1] < 0 or mutare[1] >= Joc.NR_COLOANE):
                                continue
                            if(self.tabla_joc.matr[mutare[0]][mutare[1]] == Joc.GOL):
                                lista.append(mutare)
                elif(self.tabla_joc.matr[poz[0]][poz[1]] == culoareR):
                    directii = [(1, 1), (-1, -1), (1, -1), (-1, 1)]
                    for directie in directii:
                        mutare = (poz[0]+directie[0],poz[1]+directie[1])
                        if (mutare[0] < 0 or mutare[0] >= Joc.NR_LINII or mutare[1] < 0 or mutare[1] >= Joc.NR_COLOANE):
                            continue
                        if(self.tabla_joc.matr[mutare[0]][mutare[1]] == Joc.GOL):
                            lista.append(mutare)
  
        return lista

    def sarituri(self):
        listaf = []
        listai = []
        culoareP = self.j_curent
        culoareR = chr(ord(culoareP)-32)
        
        if(culoareP == 'n'):
            
            for i in range(Joc.NR_LINII):
                for j in range(Joc.NR_COLOANE):
                    poz = (i,j)
                    if(self.tabla_joc.matr[poz[0]][poz[1]] == culoareP ):
                        perechi_directii = [(-1, -1),(-1, 1)]
                        for directie in perechi_directii:
                                mutare = (poz[0]+directie[0],poz[1]+directie[1])
                                if (mutare[0] < 0 or mutare[0] >= Joc.NR_LINII or mutare[1] < 0 or mutare[1] >= Joc.NR_COLOANE):
                                    continue
                                if(self.tabla_joc.matr[mutare[0]][mutare[1]] != Joc.GOL and self.tabla_joc.matr[mutare[0]][mutare[1]] != culoareP and self.tabla_joc.matr[mutare[0]][mutare[1]] != culoareR):
                                    #adica daca in diagonala e piesa opusa
                                    mutare1 = (mutare[0]+directie[0],mutare[1]+directie[1])
                                    if (mutare1[0] < 0 or mutare1[0] >= Joc.NR_LINII or mutare1[1] < 0 or mutare1[1] >= Joc.NR_COLOANE):
                                        continue
                                    if(self.tabla_joc.matr[mutare1[0]][mutare1[1]] == Joc.GOL):
                                        listai.append(poz)
                                        listaf.append(mutare1)
                                        
                    elif (self.tabla_joc.matr[poz[0]][poz[1]] == culoareR):
                        directii = [(1, 1), (-1, -1), (1, -1), (-1, 1)]
                        for directie in directii:
                            mutare = (poz[0]+directie[0],poz[1]+directie[1])
                            if (mutare[0] < 0 or mutare[0] >= Joc.NR_LINII or mutare[1] < 0 or mutare[1] >= Joc.NR_COLOANE):
                                continue
                            if(self.tabla_joc.matr[mutare[0]][mutare[1]] != Joc.GOL and self.tabla_joc.matr[mutare[0]][mutare[1]] != culoareP and self.tabla_joc.matr[mutare[0]][mutare[1]] != culoareR):
                                #adica daca in diagonala e piesa opusa
                                mutare1 = (mutare[0]+directie[0],mutare[1]+directie[1])
                                if (mutare1[0] < 0 or mutare1[0] >= Joc.NR_LINII or mutare1[1] < 0 or mutare1[1] >= Joc.NR_COLOANE):
                                    continue
                                if(self.tabla_joc.matr[mutare1[0]][mutare1[1]] == Joc.GOL):
                                    mutat = True
                                    listai.append(poz)
                                    listaf.append(mutare1)
        elif(culoareP == 'a'):
            
            for i in range(Joc.NR_LINII):
                for j in range(Joc.NR_COLOANE):
                    poz = (i,j)
                    if(self.tabla_joc.matr[poz[0]][poz[1]] == culoareP):
                        directii = [(1,1),(1,-1)]
                        for directie in directii:
                                mutare = (poz[0]+directie[0],poz[1]+directie[1])
                                if (mutare[0] < 0 or mutare[0] >= Joc.NR_LINII or mutare[1] < 0 or mutare[1] >= Joc.NR_COLOANE):
                                    continue
                                if(self.tabla_joc.matr[mutare[0]][mutare[1]] != Joc.GOL and self.tabla_joc.matr[mutare[0]][mutare[1]] != culoareP and self.tabla_joc.matr[mutare[0]][mutare[1]] != culoareR):
                                    #adica daca in diagonala e piesa opusa
                                    mutare1 = (mutare[0]+directie[0],mutare[1]+directie[1])
                                    if (mutare1[0] < 0 or mutare1[0] >= Joc.NR_LINII or mutare1[1] < 0 or mutare1[1] >= Joc.NR_COLOANE):
                                        continue
                                    if(self.tabla_joc.matr[mutare1[0]][mutare1[1]] == Joc.GOL):
                                        listai.append(poz)
                                        listaf.append(mutare1)
                                        
                    elif (self.tabla_joc.matr[poz[0]][poz[1]] == culoareR):
                        directii = [(1, 1), (-1, -1), (1, -1), (-1, 1)]
                        for directie in directii:
                            mutare = (poz[0]+directie[0],poz[1]+directie[1])
                            if (mutare[0] < 0 or mutare[0] >= Joc.NR_LINII or mutare[1] < 0 or mutare[1] >= Joc.NR_COLOANE):
                                continue
                            if(self.tabla_joc.matr[mutare[0]][mutare[1]] != Joc.GOL and self.tabla_joc.matr[mutare[0]][mutare[1]] != culoareP and self.tabla_joc.matr[mutare[0]][mutare[1]] != culoareR):
                                #adica daca in diagonala e piesa opusa
                                mutare1 = (mutare[0]+directie[0],mutare[1]+directie[1])
                                if (mutare1[0] < 0 or mutare1[0] >= Joc.NR_LINII or mutare1[1] < 0 or mutare1[1] >= Joc.NR_COLOANE):
                                    continue
                                if(self.tabla_joc.matr[mutare1[0]][mutare1[1]] == Joc.GOL):
                                    mutat = True
                                    listai.append(poz)
                                    listaf.append(mutare1)   
        return (listai,listaf)

    def __str__(self):
        sir = str(self.tabla_joc) + "(Jucator curent:" + self.j_curent + ")\n"
        return sir

    def __repr__(self):
        sir = str(self.tabla_joc) + "(Jucator curent:" + self.j_curent + ")\n"
        return sir
    
""" Algoritmul MinMax """

def min_max(stare):

    if stare.adancime == 0 or stare.tabla_joc.final():
        stare.scor = stare.tabla_joc.estimeaza_scor(stare.adancime)
        return stare

    # calculez toate mutarile posibile din starea curenta
    stare.mutari_posibile = stare.mutari()

    # aplic algoritmul minimax pe toate mutarile posibile (calculand astfel subarborii lor)
    mutari_scor = [min_max(mutare) for mutare in stare.mutari_posibile]

    if stare.j_curent == Joc.JMAX:
        # daca jucatorul e JMAX aleg starea-fiica cu scorul maxim
        stare.stare_aleasa = max(mutari_scor, key=lambda x: x.scor)
    else:
        # daca jucatorul e JMIN aleg starea-fiica cu scorul minim
        stare.stare_aleasa = min(mutari_scor, key=lambda x: x.scor)
    stare.scor = stare.stare_aleasa.scor
    return stare

def alpha_beta(alpha, beta, stare):
    if stare.adancime == 0 or stare.tabla_joc.final():
        stare.scor = stare.tabla_joc.estimeaza_scor(stare.adancime)
        return stare

    if alpha > beta:
        return stare  # este intr-un interval invalid deci nu o mai procesez

    stare.mutari_posibile = stare.mutari()

    if stare.j_curent == Joc.JMAX:
        scor_curent = float("-inf")
        stare.mutari_posibile.sort(key = lambda x: x.tabla_joc.estimeaza_scor(stare.adancime), reverse=True)
        

        for mutare in stare.mutari_posibile:
            # calculeaza scorul
            stare_noua = alpha_beta(alpha, beta, mutare)

            if scor_curent < stare_noua.scor:
                stare.stare_aleasa = stare_noua
                scor_curent = stare_noua.scor
            if alpha < stare_noua.scor:
                alpha = stare_noua.scor
                if alpha >= beta:
                    break

    elif stare.j_curent == Joc.JMIN:
        scor_curent = float("inf")
        stare.mutari_posibile.sort(key=lambda x: x.tabla_joc.estimeaza_scor(stare.adancime))
        for mutare in stare.mutari_posibile:

            stare_noua = alpha_beta(alpha, beta, mutare)

            if scor_curent > stare_noua.scor:
                stare.stare_aleasa = stare_noua
                scor_curent = stare_noua.scor

            if beta > stare_noua.scor:
                beta = stare_noua.scor
                if alpha >= beta:
                    break
    stare.scor = stare.stare_aleasa.scor

    return stare

def afis_daca_final(stare_curenta):
    final = stare_curenta.tabla_joc.final()
    if final:
        if final == "remiza":
            print("Remiza!")
        else:
            print("A castigat " + final)
            for linie in range(stare_curenta.tabla_joc.NR_LINII):
                for coloana in range(stare_curenta.tabla_joc.NR_COLOANE):
                    
                    if final == 'a' or final == 'A':
                        if stare_curenta.tabla_joc.matr[linie][coloana] == "a":
                            culoare = (242, 50, 29) #daca selectam piesa patratul se coloreaza cu gri
                            pygame.draw.rect(stare_curenta.tabla_joc.__class__.display, culoare, stare_curenta.tabla_joc.__class__.celuleGrid[linie * stare_curenta.tabla_joc.__class__.NR_COLOANE + coloana])  # alb = (255,255,255)
                            stare_curenta.tabla_joc.__class__.display.blit(stare_curenta.tabla_joc.__class__.a_img,
                                (coloana * (stare_curenta.tabla_joc.__class__.dim_celula + 1),linie * (stare_curenta.tabla_joc.__class__.dim_celula + 1)))
                        
                        elif stare_curenta.tabla_joc.matr[linie][coloana] == "A":
                            culoare = (242, 50, 29) #daca selectam piesa patratul se coloreaza cu gri
                            pygame.draw.rect(stare_curenta.tabla_joc.__class__.display, culoare, stare_curenta.tabla_joc.__class__.celuleGrid[linie * stare_curenta.tabla_joc.__class__.NR_COLOANE + coloana])  # alb = (255,255,255)
                            stare_curenta.tabla_joc.__class__.display.blit(stare_curenta.tabla_joc.__class__.ra_img, 
                                (coloana *(stare_curenta.tabla_joc.__class__.dim_celula+1),linie * (stare_curenta.tabla_joc.__class__.dim_celula +1)))
                            
                            
                    elif final == 'n' or final == 'N':
                        if stare_curenta.tabla_joc.matr[linie][coloana] == "n":
                            culoare = (242, 50, 29) #daca selectam piesa patratul se coloreaza cu gri
                            pygame.draw.rect(stare_curenta.tabla_joc.__class__.display, culoare, stare_curenta.tabla_joc.__class__.celuleGrid[linie * stare_curenta.tabla_joc.__class__.NR_COLOANE + coloana])  # alb = (255,255,255)
                            stare_curenta.tabla_joc.__class__.display.blit(stare_curenta.tabla_joc.__class__.n_img,
                                (coloana * (stare_curenta.tabla_joc.__class__.dim_celula + 1), linie * (stare_curenta.tabla_joc.__class__.dim_celula + 1)))
                            
                            
                        elif stare_curenta.tabla_joc.matr[linie][coloana] == "N":
                            culoare = (242, 50, 29) #daca selectam piesa patratul se coloreaza cu gri
                            pygame.draw.rect(stare_curenta.tabla_joc.__class__.display, culoare, stare_curenta.tabla_joc.__class__.celuleGrid[linie * stare_curenta.tabla_joc.__class__.NR_COLOANE + coloana])  # alb = (255,255,255)
                            stare_curenta.tabla_joc.__class__.display.blit(stare_curenta.tabla_joc.__class__.rn_img, 
                                (coloana *(stare_curenta.tabla_joc.__class__.dim_celula+1), linie * (stare_curenta.tabla_joc.__class__.dim_celula +1)))
                            
            
            pygame.display.update()

        return True

    return False

class Buton:
    def __init__(self, display=None, left=0, top=0, w=0, h=0, culoareFundal=(53, 80, 115), culoareFundalSel=(89, 134, 194), text="", font="consolas", fontDimensiune=16, culoareText=(255, 255, 255), valoare=""):
        
        self.display = display
        self.culoareFundal = culoareFundal
        self.culoareFundalSel = culoareFundalSel
        self.text = text
        self.font = font
        self.w = w
        self.h = h
        self.selectat = False
        self.fontDimensiune = fontDimensiune
        self.culoareText = culoareText
        #creez obiectul font
        fontObj = pygame.font.SysFont(self.font, self.fontDimensiune)
        self.textRandat = fontObj.render(self.text, True, self.culoareText)
        self.dreptunghi = pygame.Rect(left, top, w, h)
        #centram textul
        self.dreptunghiText = self.textRandat.get_rect(center=self.dreptunghi.center)
        self.valoare = valoare

    def selecteaza(self, sel):
        self.selectat = sel
        self.deseneaza()

    def selecteazaDupacoord(self, coord):
        if self.dreptunghi.collidepoint(coord):
            self.selecteaza(True)
            return True
        return False

    def updateDreptunghi(self):
        self.dreptunghi.left = self.left
        self.dreptunghi.top = self.top
        self.dreptunghiText = self.textRandat.get_rect(center=self.dreptunghi.center)

    def deseneaza(self):
        culoareF = self.culoareFundalSel if self.selectat else self.culoareFundal
        pygame.draw.rect(self.display, culoareF, self.dreptunghi)
        self.display.blit(self.textRandat, self.dreptunghiText)



class GrupButoane:
    def __init__(self, listaButoane=[], indiceSelectat=0, spatiuButoane=10, left=0, top=0):
        
        self.listaButoane = listaButoane
        self.indiceSelectat = indiceSelectat
        self.listaButoane[self.indiceSelectat].selectat = True
        self.top = top
        self.left = left
        leftCurent = self.left
        for b in self.listaButoane:
            b.top = self.top
            b.left = leftCurent
            b.updateDreptunghi()
            leftCurent += spatiuButoane + b.w

    def selecteazaDupacoord(self, coord):
        for ib, b in enumerate(self.listaButoane):
            if b.selecteazaDupacoord(coord):
                self.listaButoane[self.indiceSelectat].selecteaza(False)
                self.indiceSelectat = ib
                return True
        return False

    def deseneaza(self):
        # atentie, nu face wrap
        for b in self.listaButoane:
            b.deseneaza()

    def getValoare(self):
        return self.listaButoane[self.indiceSelectat].valoare
    
########## ecran initial #############

def deseneaza_alegeri(display, tabla_curenta):
    btn_alg = GrupButoane(
        top=30,
        left=30,
        listaButoane=[
            Buton(display=display, w=80, h=30, text="minimax", valoare="minimax"),
            Buton(display=display, w=80, h=30, text="alphabeta", valoare="alphabeta"),
        ],
        indiceSelectat=1,
    )
    btn_juc = GrupButoane(
        top=100,
        left=30,
        listaButoane=[
            Buton(display=display, w=50, h=30, text="alb", valoare="a"),
            Buton(display=display, w=50, h=30, text="negru", valoare="n"),
        ],
        indiceSelectat=0,
    )
   
    btn_difilcutate = GrupButoane(
        top = 170,
        left = 30,
        listaButoane=[
            Buton(display = display, w= 85,h=30,text ="incepator",valoare ="incepator"),
            Buton(display = display, w= 85,h=30,text ="mediu",valoare ="mediu"),
            Buton(display = display, w= 85,h=30,text ="avansat",valoare ="avansat"),
        ],
        indiceSelectat=0
    )
    ok = Buton(display=display, top=240, left=30, w=40, h=30, text="ok", culoareFundal=(155, 0, 55))
    btn_alg.deseneaza()
    btn_juc.deseneaza()
    btn_difilcutate.deseneaza()
    ok.deseneaza()
    
    while True:
        for ev in pygame.event.get():
            if ev.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            elif ev.type == pygame.MOUSEBUTTONDOWN:
                pos = pygame.mouse.get_pos()
                if not btn_alg.selecteazaDupacoord(pos):
                    if not btn_juc.selecteazaDupacoord(pos):
                        if not btn_difilcutate.selecteazaDupacoord(pos):
                            if ok.selecteazaDupacoord(pos):
                                tabla_curenta.deseneaza_grid()
                                return btn_juc.getValoare(), btn_alg.getValoare(),btn_difilcutate.getValoare()
        pygame.display.update()


def main():
    #setam interfata grafica
    pygame.init()
    pygame.display.set_caption("Silistru Delia-Stefania - Dame")
    nl = 8
    nc = 8
    w = 80

    ecran = pygame.display.set_mode(size=(nc * (w + 1) - 1, nl * (w + 1) - 1))  # N *100+ N-1= N*(100+1)-1
    Joc.initializeaza(ecran, NR_LINII=nl, NR_COLOANE=nc, dim_celula=w)

    # initializare tabla
    tabla_curenta = Joc()
    Joc.JMIN, tip_algoritm, dificultate = deseneaza_alegeri(ecran, tabla_curenta)
    
    if(dificultate=="incepator"):
        ADANCIME_MAX = 2
    elif(dificultate=="mediu"):
        ADANCIME_MAX = 5
    else:
        ADANCIME_MAX = 8
        
    print(Joc.JMIN, tip_algoritm)

    
    print(ADANCIME_MAX)
    # JMAx = calculator jMIN = player
    Joc.JMAX = 'a' if Joc.JMIN == 'n' else 'n'

    print("Tabla initiala:")
    print(str(tabla_curenta))

    # creare stare initiala
    # jucatorul cu piese negre incepe mereu
    stare_curenta = Stare(tabla_curenta, 'n', ADANCIME_MAX)

    de_mutat = False
    
    tabla_curenta.deseneaza_grid()
    while True:
        if stare_curenta.j_curent == Joc.JMIN:
            if(Joc.JMIN == "n"):
                linie_rege = 0
            else:
                linie_rege = 7

            obligat = stare_curenta.sarituri() # o sa fac o lista cu doua liste in prima lista avem piesa care face mutarea si a doua o sa aiba pozitia pe care ajunge
            
            for event in pygame.event.get():
                
                if event.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()
                    
                elif event.type == pygame.MOUSEBUTTONDOWN:
                    pos = pygame.mouse.get_pos()  # coordonatele cursorului

                    for np in range(len(Joc.celuleGrid)):
                        
                        if Joc.celuleGrid[np].collidepoint(pos):
                            linie = np // Joc.NR_LINII
                            coloana = np % Joc.NR_COLOANE
                            
                            if stare_curenta.tabla_joc.matr[linie][coloana] == Joc.JMIN or stare_curenta.tabla_joc.matr[linie][coloana] == chr(ord(Joc.JMIN)-32):
                                if(de_mutat and linie == de_mutat[0] and coloana == de_mutat[1]):
                                    de_mutat= False
                                    stare_curenta.tabla_joc.deseneaza_grid()
                                else:
                                    de_mutat = (linie,coloana)
                                    #desenez gridul cu patratelul marcat
                                    stare_curenta.tabla_joc.deseneaza_grid(de_mutat)
                            if stare_curenta.tabla_joc.matr[linie][coloana] == Joc.GOL:
                                if de_mutat:
                                    ### teste legate de mutarea simbolului
                                    ## bun deci aici vom avea o functie care ne va return tupluri de mutari(pozitii) pe care le poate face piesa marcata
                                    optiuni = stare_curenta.options(de_mutat)
                                    
                                    if(obligat != ([],[])):
                                        try:
                                            index = obligat[1].index((linie,coloana))
                                        except:
                                            index = 0

                                        if((linie,coloana) == obligat[1][index] and (linie,coloana) in optiuni and de_mutat == obligat[0][index]):
                                            salt = False
                                            if(de_mutat[0]+2 == linie and de_mutat[1]+2 == coloana):
                                                stare_curenta.tabla_joc.matr[de_mutat[0]+1][de_mutat[1]+1] = Joc.GOL
                                                salt = True
                                            elif(de_mutat[0]+2 == linie and de_mutat[1]-2 == coloana):
                                                stare_curenta.tabla_joc.matr[de_mutat[0]+1][de_mutat[1]-1] = Joc.GOL
                                                salt = True
                                            elif(de_mutat[0]-2 == linie and de_mutat[1]-2 == coloana):
                                                stare_curenta.tabla_joc.matr[de_mutat[0]-1][de_mutat[1]-1] = Joc.GOL
                                                salt = True
                                            elif(de_mutat[0]-2 == linie and de_mutat[1]+2 == coloana):
                                                stare_curenta.tabla_joc.matr[de_mutat[0]-1][de_mutat[1]+1] = Joc.GOL
                                                salt = True
                                            if(linie_rege == linie):
                                                stare_curenta.tabla_joc.matr[linie][coloana] = chr(ord(Joc.JMIN)-32)
                                                transformat = True
                                            else:
                                                stare_curenta.tabla_joc.matr[linie][coloana] = stare_curenta.tabla_joc.matr[de_mutat[0]][de_mutat[1]]
                                                transformat = False
                                            stare_curenta.tabla_joc.matr[de_mutat[0]][de_mutat[1]] = Joc.GOL
                                            stare_curenta.tabla_joc.ultima_mutare = (linie,coloana)
                                            de_mutat=False
                                            # afisarea starii jocului in urma mutarii utilizatorului
                                            print("\nTabla dupa mutarea jucatorului")
                                            print(str(stare_curenta))

                                            stare_curenta.tabla_joc.deseneaza_grid()
                                            # testez daca jocul a ajuns intr-o stare finala
                                            # si afisez un mesaj corespunzator in caz ca da
                                            if afis_daca_final(stare_curenta):
                                                break

                                            # S-a realizat o mutare. Schimb jucatorul cu cel opus
                                            # aici o sa apara modificare pentru salt
                                            if(stare_curenta.sarituri() == ([],[]) or transformat ):
                                                stare_curenta.j_curent = Joc.jucator_opus(stare_curenta.j_curent)
                                                
                                    elif((linie,coloana) in optiuni):
                                        if(linie_rege == linie):
                                            stare_curenta.tabla_joc.matr[linie][coloana] = chr(ord(Joc.JMIN)-32)

                                        else:
                                            stare_curenta.tabla_joc.matr[linie][coloana] = stare_curenta.tabla_joc.matr[de_mutat[0]][de_mutat[1]]

                                        stare_curenta.tabla_joc.matr[de_mutat[0]][de_mutat[1]] = Joc.GOL
                                        stare_curenta.tabla_joc.ultima_mutare = (linie,coloana)
                                        de_mutat=False
                                        # afisarea starii jocului in urma mutarii utilizatorului
                                        print("\nTabla dupa mutarea jucatorului")
                                        print(str(stare_curenta))

                                        stare_curenta.tabla_joc.deseneaza_grid()
                                        # testez daca jocul a ajuns intr-o stare finala
                                        # si afisez un mesaj corespunzator in caz ca da
                                        if afis_daca_final(stare_curenta):
                                            break

                                        # S-a realizat o mutare. Schimb jucatorul cu cel opus
                                        # aici o sa apara modificare pentru salt
                                        stare_curenta.j_curent = Joc.jucator_opus(
                                        stare_curenta.j_curent
                                        )

        # --------------------------------
        else:  # jucatorul e JMAX (calculatorul)
            # Mutare calculator

            # preiau timpul in milisecunde de dinainte de mutare
            t_inainte = int(round(time.time() * 1000))
            if tip_algoritm == "minimax":
                stare_actualizata = min_max(stare_curenta)
            else:  # tip_algoritm=="alphabeta"
                stare_actualizata = alpha_beta(-500, 500, stare_curenta)
            k = 1
        
            if(stare_curenta.sarituri() == ([],[])):
                k = 0
            stare_curenta.tabla_joc = stare_actualizata.stare_aleasa.tabla_joc

            print("Tabla dupa mutarea calculatorului\n" + str(stare_curenta))

            # preiau timpul in milisecunde de dupa mutare
            t_dupa = int(round(time.time() * 1000))
            print(
                'Calculatorul a "gandit" timp de '
                + str(t_dupa - t_inainte)
                + " milisecunde."
            )

            stare_curenta.tabla_joc.deseneaza_grid()
            if afis_daca_final(stare_curenta):
                break
            rege = False
            temp = stare_curenta.tabla_joc.ultima_mutare
            if (stare_curenta.j_curent == "n" and temp[0] == 0):
                rege = True
            elif(stare_curenta.j_curent == "a" and temp[0] == 1):
                rege = True

            # S-a realizat o mutare. Schimb jucatorul cu cel opus
            if(stare_curenta.sarituri() == ([],[]) or k ==0 or rege):
                stare_curenta.j_curent = Joc.jucator_opus(stare_curenta.j_curent)

   
if __name__ == "__main__":
    main()
    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()

a alphabeta
2
Tabla initiala:
  |a b c d e f g h
------------------
0 |# a # a # a # a
1 |a # a # a # a #
2 |# a # a # a # a
3 |# # # # # # # #
4 |# # # # # # # #
5 |n # n # n # n #
6 |# n # n # n # n
7 |n # n # n # n #
Tabla dupa mutarea calculatorului
  |a b c d e f g h
------------------
0 |# a # a # a # a
1 |a # a # a # a #
2 |# a # a # a # a
3 |# # # # # # # #
4 |# n # # # # # #
5 |# # n # n # n #
6 |# n # n # n # n
7 |n # n # n # n #(Jucator curent:n)

Calculatorul a "gandit" timp de 9 milisecunde.

Tabla dupa mutarea jucatorului
  |a b c d e f g h
------------------
0 |# a # a # a # a
1 |a # a # a # a #
2 |# # # a # a # a
3 |a # # # # # # #
4 |# n # # # # # #
5 |# # n # n # n #
6 |# n # n # n # n
7 |n # n # n # n #(Jucator curent:a)

Tabla dupa mutarea calculatorului
  |a b c d e f g h
------------------
0 |# a # a # a # a
1 |a # a # a # a #
2 |# # # a # a # a
3 |a # # # # # # #
4 |# n # n # # # #
5 |# # n # # # n #
6 |# n # n # n # n
7 |n # n # n # n #(Jucator curent:n)

Calcul


Tabla dupa mutarea jucatorului
  |a b c d e f g h
------------------
0 |# a # # # # # #
1 |n # # # a # a #
2 |# a # # # a # a
3 |a # n # a # a #
4 |# n # n # # # n
5 |n # n # n # a #
6 |# # # n # n # #
7 |# # # # n # # #(Jucator curent:a)

Tabla dupa mutarea calculatorului
  |a b c d e f g h
------------------
0 |# a # # # # # #
1 |n # # # a # a #
2 |# a # n # a # a
3 |a # # # a # a #
4 |# n # n # # # n
5 |n # n # n # a #
6 |# # # n # n # #
7 |# # # # n # # #(Jucator curent:n)

Calculatorul a "gandit" timp de 2 milisecunde.

Tabla dupa mutarea jucatorului
  |a b c d e f g h
------------------
0 |# a # # # # # #
1 |n # # # # # a #
2 |# a # # # a # a
3 |a # a # a # a #
4 |# n # n # # # n
5 |n # n # n # a #
6 |# # # n # n # #
7 |# # # # n # # #(Jucator curent:a)

Tabla dupa mutarea calculatorului
  |a b c d e f g h
------------------
0 |# a # # # # # #
1 |n # # # # # a #
2 |# a # n # a # a
3 |a # # # a # a #
4 |# # # n # # # n
5 |n # n # n # a #
6 |# # # n # n # #
7 |# # # # n # # #(Juca


Tabla dupa mutarea jucatorului
  |a b c d e f g h
------------------
0 |# # # # # # # N
1 |# # # # N # # #
2 |# a # # # # # N
3 |a # # # # # # #
4 |# # # n # # # n
5 |n # n # n # # #
6 |# # # n # A # a
7 |# # # # n # # #(Jucator curent:a)

Tabla dupa mutarea calculatorului
  |a b c d e f g h
------------------
0 |# # # # # # # N
1 |# # # # N # # #
2 |# a # # # # # N
3 |a # # # # # # #
4 |# # # n # # # n
5 |n # n # n # n #
6 |# # # n # # # a
7 |# # # # # # # #(Jucator curent:n)

Calculatorul a "gandit" timp de 1 milisecunde.

Tabla dupa mutarea jucatorului
  |a b c d e f g h
------------------
0 |# # # # # # # N
1 |# # # # N # # #
2 |# a # # # # # N
3 |a # # # # # # #
4 |# # # n # # # n
5 |n # n # n # n #
6 |# # # n # # # #
7 |# # # # # # A #(Jucator curent:a)

Tabla dupa mutarea calculatorului
  |a b c d e f g h
------------------
0 |# # # # # # # #
1 |# # # # N # N #
2 |# a # # # # # N
3 |a # # # # # # #
4 |# # # n # # # n
5 |n # n # n # n #
6 |# # # n # # # #
7 |# # # # # # A #(Juca